# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Visualization of a Coronavirus

In [ ]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, \
                        Cell, Sugars, Vector2, Vector3, Quaternion
be = BioExplorer('localhost:5000')
be.reset()
print(be.version())

In [ ]:
# Resources
resource_folder = '../tests/test_files/'

pdb_folder = resource_folder + 'pdb/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

In [ ]:
status = be.core_api().set_camera(
    orientation=[6.749507132163666e-17, -0.7071067811865477, -0.7071067811865474, 1.5409067694518602e-16],
    position=[-5.066518495759233, 242.54810500047284, -20.40108958115976],
    target=[-5.06651849575913, -86.22886126543523, -20.40108958115971]
)

In [ ]:
# Scene
scene_size = 400

# Proteins
protein_representation = be.REPRESENTATION_ATOMS

In [ ]:
from IPython.display import clear_output
from braynsmediamaker import MovieMaker

mm = MovieMaker(be.core_api())

name='Coronavirus'

for frame in range(101, 120, 1):
# for frame in [40]:
    clear_output()
    be.set_general_settings(model_visibility_on_creation=False, off_folder='/tmp/')
    print('Frame %d' % frame)
    name='Coronavirus'
    be.add_coronavirus(
#         nb_protein_s=1, nb_protein_e=1, nb_protein_m=1,
        name=name, resource_folder=resource_folder,
        representation=protein_representation,
#         orientation=Quaternion(0.0, 0.0, 0.707, 0.707),
        add_glycans=False,
        morphing_step=frame / 100.0,
        clipping_planes=[[0,-1,0,0]]
    )
    be.set_models_visibility(True)
#     break
    mm.create_snapshot(
        size=[960, 540], samples_per_pixel=64,
        path='/home/favreau/Videos/%05d.png' % frame)

In [ ]:
cam = be.core_api().get_camera()
print('orientation=' + str(cam['orientation']))
print('position=' + str(cam['position']))
print('target=' + str(cam['target']))

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.13276691605188376, -0.7023228724898958, -0.6879740137773086, -0.12572702631794422],
    position=[157.8095734962004, 275.3477113580235, -38.65013309466292],
    target=[5.120867584888782, -121.15036462213698, -29.398712780300983]
)

In [ ]:
# ACE2 receptor definition
ace2_receptor = Protein(
    sources=[pdb_folder + '6m18.pdb'],
    occurences=1,
    position=Vector3(0.0, 6.0, 0.0))

# Membrane definition
membrane_size = scene_size
membrane_height = 80
membrane = Membrane(
    sources=[pdb_folder + 'membrane/popc.pdb'],
    occurences=80000)

# Cell definition
name='Cell'
cell = Cell(
    name=name,
    size=Vector2(membrane_size, membrane_height),
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    membrane=membrane, receptor=ace2_receptor,
)

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=Vector3(0.0, 0.0, -56.0), orientation=Quaternion(0.707,0.707,0,0),
    random_seed=1
)

status = be.set_protein_color_scheme(
    name, name + '_' + be.NAME_RECEPTOR, 
    be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)

## Rendering settings

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5000
params.shadows = 1.0
params.soft_shadows = 0.1
params.fog_start = 1300
params.fog_thickness = 1300
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_PERLIN,
    user_parameter=0.03, specular_exponent=5, glossiness=0.1)